<a href="https://colab.research.google.com/github/Revoluation/Function-Calling/blob/main/functionCalling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain langchain_community langchain_google_genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 5.4 MB/s eta 0:00:00


In [4]:
from google.colab import userdata
import os
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

In [6]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model='gemini-1.5-flash',verbose=True)
llm.invoke("Hi")

AIMessage(content='Hi there! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-12a0a6eb-d757-451e-96ee-60c4c02da32c-0', usage_metadata={'input_tokens': 2, 'output_tokens': 11, 'total_tokens': 13, 'input_token_details': {'cache_read': 0}})

In [7]:
from langchain_core.tools import tool
@tool
def multiply(a:int, b:int)-> int:
  """multiply both numbers.
    Args:
     a integer as first number
     b integer as second number    """
  return a*b

tools_list = [multiply]
model_with_tools = llm.bind_tools(tools_list)

result = model_with_tools.invoke('what is 2 multiply by 10')
result

AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 2.0, "b": 10.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-918dbe95-6142-40d1-9c57-3f43b45d2d2e-0', tool_calls=[{'name': 'multiply', 'args': {'a': 2.0, 'b': 10.0}, 'id': '9fae2b46-41bc-489c-bde6-9af015b54174', 'type': 'tool_call'}], usage_metadata={'input_tokens': 73, 'output_tokens': 3, 'total_tokens': 76, 'input_token_details': {'cache_read': 0}})

In [9]:
from langchain.tools import Tool
def add (a:int, b: int) -> int:
  return a + b

math_tools = Tool(
    name = 'AdditionTool',
    func = lambda x: f"The result is : {add(*map(int, x.split()))}",
    description="Adds two numbers. Input should be two integers separated by a space."

)

In [10]:
from langchain.agents import initialize_agent ,AgentType
tools = [math_tools]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

<ipython-input-10-6f434a375ff1>:4: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [11]:
my_response = agent.run('what is 30 + 99')
print(my_response)

<ipython-input-11-7a73d5b62c66>:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  my_response = agent.run('what is 30 + 99')




> Entering new AgentExecutor chain...
Thought: I need to add 30 and 99.  I can use the AdditionTool for this.
Action: AdditionTool
Action Input: 30 99
Observation: The result is : 129
Thought:Thought: I now know the final answer
Final Answer: 129

> Finished chain.
129
